# SF3D API Server - JupyterHub (v2)

This notebook runs a FastAPI server on the school GPU to generate 3D meshes using Stable Fast 3D.

## ⚠️ IMPORTANT: Run cells in order from top to bottom!

**Setup Steps:**
1. Run Cell 1: Install dependencies (wait for completion)
2. **RESTART KERNEL** (Kernel → Restart)
3. Run Cell 2: Import libraries
4. Run Cell 3: Load SF3D model (takes 10-30 seconds first time)
5. Run Cell 4: Define API endpoints
6. Run Cell 5: Start server (keep running)
7. Test from Mac: `python tests/sf3d_api_client.py <image_path>`

**Server URL:** `http://itp-ml.itp.tsoa.nyu.edu:<PORT>/`

## Cell 1: Install Dependencies

**Run this first, then RESTART KERNEL before continuing!**

In [ ]:
%%bash
# Install FastAPI and web server
pip install --user fastapi uvicorn[standard] python-multipart

# Install PyTorch with CUDA 11.8 (matching server)
pip install --user torch torchvision --index-url https://download.pytorch.org/whl/cu118

# Install 3D and image libraries
pip install --user trimesh pillow numpy

# Install transformers for model loading
pip install --user transformers accelerate

# Install Stability AI's TripoSR (similar to SF3D, well-supported)
pip install --user git+https://github.com/VAST-AI-Research/TripoSR.git

echo ""
echo "✅ Installation complete!"
echo ""
echo "⚠️  IMPORTANT: Now go to Kernel → Restart Kernel"
echo "   Then run the remaining cells in order."

## Cell 2: Import Libraries

**After restarting kernel, run this cell**

In [ ]:
import io
import os
import time
from pathlib import Path
from typing import Optional

import torch
import numpy as np
from PIL import Image
from fastapi import FastAPI, File, UploadFile, Form, HTTPException
from fastapi.responses import FileResponse, JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import uvicorn

print("✅ All libraries imported successfully")
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️  WARNING: CUDA not available! Will use CPU (very slow)")

## Cell 3: Load TripoSR Model

**This loads the 3D generation model (first time: 10-30 seconds)**

In [ ]:
# Configure device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load TripoSR model
print("\nLoading TripoSR model...")
print("(First time: downloads ~2GB model, may take 30 seconds)")
start_time = time.time()

try:
    from tsr.system import TSR
    
    model = TSR.from_pretrained(
        "stabilityai/TripoSR",
        config_name="config.yaml",
        weight_name="model.ckpt",
    )
    model.renderer.set_chunk_size(8192)  # Optimize for GPU
    model.to(device)
    
    print(f"\n✅ TripoSR model loaded in {time.time() - start_time:.1f}s")
    
except Exception as e:
    print(f"\n❌ Error loading TripoSR: {e}")
    print("\nTrying alternative method...")
    
    # Fallback: Manual implementation
    import sys
    sys.path.insert(0, '/home/jovyan/.local/lib/python3.10/site-packages')
    
    from tsr.system import TSR
    model = TSR.from_pretrained("stabilityai/TripoSR")
    model.to(device)
    
    print(f"\n✅ Model loaded (fallback) in {time.time() - start_time:.1f}s")

# Create output directory
output_dir = Path("/tmp/sf3d_outputs")
output_dir.mkdir(exist_ok=True)
print(f"\nOutput directory: {output_dir}")
print("\n✅ Ready to generate meshes!")

## Cell 4: Define API Endpoints

In [ ]:
# Initialize FastAPI
app = FastAPI(
    title="TripoSR 3D Generation API",
    description="Generate 3D meshes from images using TripoSR",
    version="2.0.0"
)

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/")
async def root():
    return {
        "message": "TripoSR 3D Generation API",
        "status": "running",
        "device": device,
        "model": "stabilityai/TripoSR",
        "endpoints": {
            "/generate": "POST - Generate 3D mesh from image",
            "/health": "GET - Health check"
        }
    }

@app.get("/health")
async def health():
    return {
        "status": "healthy",
        "device": device,
        "cuda_available": torch.cuda.is_available(),
        "model_loaded": model is not None
    }

@app.post("/generate")
async def generate_mesh(
    file: UploadFile = File(...),
    texture_resolution: int = Form(1024),
    remesh_option: str = Form("none"),
    foreground_ratio: float = Form(0.85)
):
    """
    Generate 3D mesh from uploaded image.
    
    Parameters:
    - file: Image file (PNG, JPG)
    - texture_resolution: Not used (TripoSR auto)
    - remesh_option: Not used (TripoSR auto)
    - foreground_ratio: Not used (TripoSR auto)
    
    Returns:
    - GLB file
    """
    try:
        # Read and validate image
        image_data = await file.read()
        image = Image.open(io.BytesIO(image_data))
        
        # Convert to RGB if needed
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        print(f"\n[{time.strftime('%H:%M:%S')}] Received image: {image.size}")
        
        # Generate mesh
        start_time = time.time()
        
        with torch.no_grad():
            # TripoSR inference
            scene_codes = model([image], device=device)
            
            # Extract mesh
            meshes = model.extract_mesh(scene_codes)
            mesh = meshes[0]
        
        generation_time = time.time() - start_time
        print(f"[{time.strftime('%H:%M:%S')}] Generation completed in {generation_time:.2f}s")
        
        # Save mesh to temporary file
        timestamp = int(time.time() * 1000)
        output_path = output_dir / f"mesh_{timestamp}.glb"
        
        # Export as GLB
        mesh.export(str(output_path), file_type='glb')
        
        file_size = output_path.stat().st_size
        print(f"[{time.strftime('%H:%M:%S')}] Saved: {output_path.name} ({file_size / 1024:.1f} KB)\n")
        
        # Return file
        return FileResponse(
            path=output_path,
            media_type="model/gltf-binary",
            filename=f"mesh_{timestamp}.glb",
            headers={
                "X-Generation-Time": str(generation_time),
                "X-File-Size": str(file_size)
            }
        )
    
    except Exception as e:
        print(f"\n❌ Error generating mesh: {e}")
        import traceback
        traceback.print_exc()
        raise HTTPException(status_code=500, detail=str(e))

print("\n✅ API endpoints defined")
print("   Ready to start server!")

## Cell 5: Start Server

**⚠️ IMPORTANT:**
- This cell will run indefinitely
- Keep the notebook open
- Press ■ (stop) button to stop server
- Check if port 8765 is available (change if needed)

In [ ]:
# Configure server
PORT = 8765  # Change this if port is already in use (try 8766, 8767, etc.)
HOST = "0.0.0.0"  # Listen on all interfaces

print("="*70)
print("🚀 Starting TripoSR API Server")
print("="*70)
print(f"Server URL: http://itp-ml.itp.tsoa.nyu.edu:{PORT}/")
print(f"Device: {device}")
print(f"Model: TripoSR")
print(f"")
print("Available endpoints:")
print(f"  GET  http://itp-ml.itp.tsoa.nyu.edu:{PORT}/           - API info")
print(f"  GET  http://itp-ml.itp.tsoa.nyu.edu:{PORT}/health     - Health check")
print(f"  POST http://itp-ml.itp.tsoa.nyu.edu:{PORT}/generate   - Generate 3D mesh")
print(f"")
print("To test from your Mac:")
print(f"  python tests/sf3d_api_client.py <image> --server http://itp-ml.itp.tsoa.nyu.edu:{PORT}")
print("")
print("Or test with curl:")
print(f"  curl http://itp-ml.itp.tsoa.nyu.edu:{PORT}/health")
print("="*70)
print("")
print("⚠️  KEEP THIS CELL RUNNING - Server logs will appear below")
print("")

# Run server
try:
    uvicorn.run(app, host=HOST, port=PORT, log_level="info")
except KeyboardInterrupt:
    print("\n\n✅ Server stopped")
except Exception as e:
    print(f"\n\n❌ Server error: {e}")
    print("\nTroubleshooting:")
    print("  - If port is in use, change PORT above and re-run this cell")
    print("  - If permission denied, try PORT > 1024")

---

## Troubleshooting

### "Port already in use"
- Change `PORT = 8765` to another number (8766, 8767, etc.)
- Re-run Cell 5 only

### "CUDA out of memory"
- Someone else may be using the GPU
- Try again later or use smaller images

### "Module not found" errors
- Go back to Cell 1
- Re-run installation
- RESTART KERNEL (Kernel → Restart)
- Run cells 2-5 again

### Server not accessible from Mac
- Make sure you're on NYU network/VPN
- Check server URL matches what's printed above
- Test with curl first: `curl http://itp-ml.itp.tsoa.nyu.edu:8765/health`